In [1]:
#!pip install compiler_gym 'ray[default,rllib]' &>/dev/null || echo "Install failed!"

import compiler_gym
import ray

from ray.rllib.agents.ppo import PPOTrainer
from compiler_gym.wrappers import ConstrainedCommandline, TimeLimit
from ray import tune
from itertools import islice
from compiler_gym.wrappers import CycleOverBenchmarks
from compiler_gym.util.registration import register

import loop_tool_service

from service_py.datasets import loop_tool_dataset
from service_py.rewards import flops_loop_nest_reward, flops_reward, runtime_reward
import wandb
wandb.init(project="loop_tool", entity="dejang", sync_tensorboard=True)

/home/dejang/anaconda3/envs/compiler_gym/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dejang. Use `wandb login --relogin` to force relogin


In [2]:
def make_env() -> compiler_gym.envs.CompilerEnv:
    """Make the reinforcement learning environment for this experiment."""
    
    env = loop_tool_service.make(
        "loop_tool_env-v0",
        observation_space="stride_tensor",
        reward_space="flops_loop_nest_tensor",
        # reward_space="runtime",
    )

    env = TimeLimit(env, max_episode_steps=10)
    return env

In [3]:
with make_env() as env:
    print("Action space:", env.action_space)
    print("Observation space:", env.observation_space)
    print("Reward space:", env.reward_space)

Action space: NamedDiscrete([up, down, swap_up, swap_down])
Observation space: Box([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]], [[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
  inf inf inf inf inf inf inf inf inf inf inf inf inf inf]], (1, 32), float32)
Reward space: flops_loop_nest_tensor


In [4]:
with make_env() as env:
    # The two datasets we will be using:
    lt_dataset = env.datasets["loop_tool_simple-v0"]
    # train_benchmarks = list(islice(lt_dataset.benchmarks(), 1))
    # test_benchmarks = list(islice(lt_dataset.benchmarks(), 2))
    
    bench = ["benchmark://loop_tool_simple-v0/simple"]
            #  "benchmark://loop_tool_simple-v0/mm128", 
            #  "benchmark://loop_tool_simple-v0/mm"] 

    train_benchmarks = bench
    test_benchmarks = bench

print("Number of benchmarks for training:", len(train_benchmarks))
print("Number of benchmarks for testing:", len(test_benchmarks))


Number of benchmarks for training: 1
Number of benchmarks for testing: 1


In [5]:
def make_training_env(*args) -> compiler_gym.envs.CompilerEnv:
    """Make a reinforcement learning environment that cycles over the
    set of training benchmarks in use.
    """
    del args  # Unused env_config argument passed by ray
    return CycleOverBenchmarks(make_env(), train_benchmarks)


In [6]:
with make_training_env() as env:
    env.reset()
    print(env.benchmark)
    env.reset()
    print(env.benchmark)

for m_5586 in 128 : L0  <<<<<< cursor (line 0 )
 for n_5625 in 128 : L1  
  for k_5587 in 128 : L2  
   %2[m_5586, k_5587, n_5625] <- multiply(%0, %1)  
   %3[m_5586, n_5625] <- add(%2)  
  %4[m_5586, n_5625] <- write(%3)  

benchmark://loop_tool_simple-v0/simple
for m_5586 in 128 : L0  <<<<<< cursor (line 0 )
 for n_5625 in 128 : L1  
  for k_5587 in 128 : L2  
   %2[m_5586, k_5587, n_5625] <- multiply(%0, %1)  
   %3[m_5586, n_5625] <- add(%2)  
  %4[m_5586, n_5625] <- write(%3)  



E0629 14:54:21.600706 139853277578816 example_service.py:249] CRITICAL - 

Working_dir = /dev/shm/compiler_gym_dejang/s/0629T145420-561928-1422

E0629 14:54:21.708813 139853277578816 example_service.py:249] CRITICAL - 

Working_dir = /dev/shm/compiler_gym_dejang/s/0629T145420-561928-1422



benchmark://loop_tool_simple-v0/simple


In [7]:
env = make_training_env()
env.reset()

for m_5586 in 128 : L0  <<<<<< cursor (line 0 )
 for n_5625 in 128 : L1  
  for k_5587 in 128 : L2  
   %2[m_5586, k_5587, n_5625] <- multiply(%0, %1)  
   %3[m_5586, n_5625] <- add(%2)  
  %4[m_5586, n_5625] <- write(%3)  



E0629 14:54:23.185833 140463474468416 example_service.py:249] CRITICAL - 

Working_dir = /dev/shm/compiler_gym_dejang/s/0629T145422-151049-0111



array([[2113536.,       0.,       0.,       0.,       0.,       0.,
              0., 2097280.,       0.,       0.,       0.,       0.,
              0.,       0.,       0.,       0.,       0.,       0.,
              0.,       0.,       0.,       0.,       0.,       0.,
              0.,       0.,       0.,       0.,       0.,       0.,
              0.,       0.]], dtype=float32)

In [8]:
env.step(2)

(array([[2113536.,       0.,       0.,       0.,       0.,       0.,
               0., 2097280.,       0.,       0.,       0.,       0.,
               0.,       0.,       0.,       0.,       0.,       0.,
               0.,       0.,       0.,       0.,       0.,       0.,
               0.,       0.,       0.,       0.,       0.,       0.,
               0.,       0.]], dtype=float32),
 -2144890833.176635,
 False,
 {'action_had_no_effect': True, 'new_action_space': False})

In [9]:
if ray.is_initialized():
    ray.shutdown()
ray.init(include_dashboard=False, ignore_reinit_error=True)

tune.register_env("compiler_gym", make_training_env)

In [10]:
import time
from ray import tune
from ray.tune import Stopper

class TimeStopper(Stopper):
    def __init__(self):
        self._start = time.time()
        self._deadline = 1

    def __call__(self, trial_id, result):
        return False

    def stop_all(self):
        return time.time() - self._start > self._deadline


In [11]:
analysis = tune.run(
    PPOTrainer,
    checkpoint_at_end=True,
    stop={
        "episodes_total": 5,
    },
    config={
        "seed": 0xCC,
        "num_workers": 1,
        # Specify the environment to use, where "compiler_gym" is the name we 
        # passed to tune.register_env().
        "env": "compiler_gym",
        # Reduce the size of the batch/trajectory lengths to match our short 
        # training run.
        "rollout_fragment_length": 5,
        "train_batch_size": 5,
        "sgd_minibatch_size": 5,
        "model": {                            # The NN model we'll optimize.
            'fcnet_hiddens': [                # "Fully-connected network with N hidden layers".
                tune.grid_search([20, 40]),   # Try these four values for layer one.
                tune.grid_search([20, 40])    # Try these four values for layer two.
            ]
        },
    }
)

wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /home/dejang/ray_results/PPOTrainer_2022-06-29_14-54-26/PPOTrainer_compiler_gym_e618f_00000_0_0=20,1=20_2022-06-29_14-54-26
(PPOTrainer pid=2747313) 2022-06-29 14:54:30,987	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPOTrainer pid=2747313) 2022-06-29 14:54:31,194	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=2747313) 2022-06-29 14:54:31,194	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=2747365) E0629 

(RolloutWorker pid=2747365) for m_5586 in 128 : L0  <<<<<< cursor (line 0 )
(RolloutWorker pid=2747365)  for n_5625 in 128 : L1  
(RolloutWorker pid=2747365)   for k_5587 in 128 : L2  
(RolloutWorker pid=2747365)    %2[m_5586, k_5587, n_5625] <- multiply(%0, %1)  
(RolloutWorker pid=2747365)    %3[m_5586, n_5625] <- add(%2)  
(RolloutWorker pid=2747365)   %4[m_5586, n_5625] <- write(%3)  
(RolloutWorker pid=2747365) 


Trial name,status,loc,model/fcnet_hiddens/0,model/fcnet_hiddens/1
PPOTrainer_compiler_gym_e618f_00000,RUNNING,100.37.253.28:2747313,20,20
PPOTrainer_compiler_gym_e618f_00001,PENDING,,40,20
PPOTrainer_compiler_gym_e618f_00002,PENDING,,20,40
PPOTrainer_compiler_gym_e618f_00003,PENDING,,40,40


(PPOTrainer pid=2747313) 2022-06-29 14:54:39,220	WARNING util.py:65 -- Install gputil for GPU system monitoring.
wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`
wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /home/dejang/ray_results/PPOTrainer_2022-06-29_14-54-26/PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39
(RolloutWorker pid=2747365) E0629 14:54:39.249601 139877103203904 example_service.py:249] CRITICAL - 
(RolloutWorker pid=2747365) 
(RolloutWorker pid=2747365) Working_dir = /dev/shm/compiler_gym_dejang/s/0629T145435-554058-6f35
(RolloutWorker pid=2747365) 


(RolloutWorker pid=2747365) for m_5586 in 128 : L0  <<<<<< cursor (line 0 )
(RolloutWorker pid=2747365)  for n_5625 in 128 : L1  
(RolloutWorker pid=2747365)   for k_5587 in 128 : L2  
(RolloutWorker pid=2747365)    %2[m_5586, k_5587, n_5625] <- multiply(%0, %1)  
(RolloutWorker pid=2747365)    %3[m_5586, n_5625] <- add(%2)  
(RolloutWorker pid=2747365)   %4[m_5586, n_5625] <- write(%3)  
(RolloutWorker pid=2747365) 
(RolloutWorker pid=2747365) Action = down
(RolloutWorker pid=2747365) for m_5586 in 128 : L0  <<<<<< cursor (line 0 )
(RolloutWorker pid=2747365)  for n_5625 in 128 : L1  
(RolloutWorker pid=2747365)   for k_5587 in 128 : L2  
(RolloutWorker pid=2747365)    %2[m_5586, k_5587, n_5625] <- multiply(%0, %1)  
(RolloutWorker pid=2747365)    %3[m_5586, n_5625] <- add(%2)  
(RolloutWorker pid=2747365)   %4[m_5586, n_5625] <- write(%3)  
(RolloutWorker pid=2747365) 
(RolloutWorker pid=2747365) >>> AGENT ITERATION =  1
(RolloutWorker pid=2747365) Action = down
(RolloutWorker pid=27

...

(RolloutWorker pid=2747668) for m_5586 in 128 : L0  <<<<<< cursor (line 0 )
(RolloutWorker pid=2747668)  for n_5625 in 128 : L1  
(RolloutWorker pid=2747668)   for k_5587 in 128 : L2  
(RolloutWorker pid=2747668)    %2[m_5586, k_5587, n_5625] <- multiply(%0, %1)  
(RolloutWorker pid=2747668)    %3[m_5586, n_5625] <- add(%2)  
(RolloutWorker pid=2747668)   %4[m_5586, n_5625] <- write(%3)  
(RolloutWorker pid=2747668) 
Result for PPOTrainer_compiler_gym_e618f_00000:
  agent_timesteps_total: 50
  counters:
    num_agent_steps_sampled: 50
    num_agent_steps_trained: 50
    num_env_steps_sampled: 50
    num_env_steps_trained: 50
  custom_metrics: {}
  date: 2022-06-29_14-55-46
  done: true
  episode_len_mean: 10.0
  episode_media: {}
  episode_reward_max: -2030386789.198473
  episode_reward_mean: -2062143252.0039055
  episode_reward_min: -2112968590.5859597
  episodes_this_iter: 1
  episodes_total: 5
  experiment_id: f88dfa7df9ca45168f205a8ba4639621
  hostname: codah
  info:
    learner:
 

(RolloutWorker pid=2748252) E0629 14:55:49.535240 139696420152896 example_service.py:249] CRITICAL - 
(RolloutWorker pid=2748252) 
(RolloutWorker pid=2748252) Working_dir = /dev/shm/compiler_gym_dejang/s/0629T145514-204240-6f35
(RolloutWorker pid=2748252) 


Trial name,status,loc,model/fcnet_hiddens/0,model/fcnet_hiddens/1,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_compiler_gym_e618f_00003,RUNNING,100.37.253.28:2748095,40,40,10,31.3922,50,-2.05893e+09,-2.02573e+09,-2.11379e+09,10
PPOTrainer_compiler_gym_e618f_00000,TERMINATED,100.37.253.28:2747313,20,20,10,31.4502,50,-2.06214e+09,-2.03039e+09,-2.11297e+09,10
PPOTrainer_compiler_gym_e618f_00001,TERMINATED,100.37.253.28:2747508,40,20,10,31.663,50,-2.05834e+09,-2.03266e+09,-2.11386e+09,10
PPOTrainer_compiler_gym_e618f_00002,TERMINATED,100.37.253.28:2747808,20,40,10,31.0087,50,-2.06312e+09,-2.02518e+09,-2.11639e+09,10


Trial name,status,loc,model/fcnet_hiddens/0,model/fcnet_hiddens/1,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_compiler_gym_e618f_00000,TERMINATED,100.37.253.28:2747313,20,20,10,31.4502,50,-2.06214e+09,-2.03039e+09,-2.11297e+09,10
PPOTrainer_compiler_gym_e618f_00001,TERMINATED,100.37.253.28:2747508,40,20,10,31.663,50,-2.05834e+09,-2.03266e+09,-2.11386e+09,10
PPOTrainer_compiler_gym_e618f_00002,TERMINATED,100.37.253.28:2747808,20,40,10,31.0087,50,-2.06312e+09,-2.02518e+09,-2.11639e+09,10
PPOTrainer_compiler_gym_e618f_00003,TERMINATED,100.37.253.28:2748095,40,40,10,31.3922,50,-2.05893e+09,-2.02573e+09,-2.11379e+09,10


2022-06-29 14:55:50,536	INFO tune.py:747 -- Total run time: 83.88 seconds (83.05 seconds for the tuning loop).


In [12]:
checkpoint = analysis.get_best_checkpoint(
    metric="episode_reward_mean",
    mode="max",
    trial=analysis.trials[0]
)

In [13]:
analysis.dataframe()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_trained,...,info/learner/default_policy/learner_stats/entropy,info/learner/default_policy/learner_stats/entropy_coeff,config/env,config/model,config/num_workers,config/rollout_fragment_length,config/seed,config/sgd_minibatch_size,config/train_batch_size,logdir
0,-2.030387e+09,-2.112969e+09,-2.062143e+09,10.0,1,1,50,50,50,50,...,1.386139,0.0,compiler_gym,"{'fcnet_hiddens': [20, 20]}",1,5,204,5,5,/home/dejang/ray_results/PPOTrainer_2022-06-29...
1,-2.032656e+09,-2.113861e+09,-2.058342e+09,10.0,1,1,50,50,50,50,...,1.385865,0.0,compiler_gym,"{'fcnet_hiddens': [40, 20]}",1,5,204,5,5,/home/dejang/ray_results/PPOTrainer_2022-06-29...
2,-2.025184e+09,-2.116393e+09,-2.063117e+09,10.0,1,1,50,50,50,50,...,1.385668,0.0,compiler_gym,"{'fcnet_hiddens': [20, 40]}",1,5,204,5,5,/home/dejang/ray_results/PPOTrainer_2022-06-29...
3,-2.025730e+09,-2.113787e+09,-2.058932e+09,10.0,1,1,50,50,50,50,...,1.384678,0.0,compiler_gym,"{'fcnet_hiddens': [40, 40]}",1,5,204,5,5,/home/dejang/ray_results/PPOTrainer_2022-06-29...


In [14]:
trial = analysis.get_best_trial(metric="episode_reward_mean", mode="max")
log_dir = analysis.get_best_logdir(metric="episode_reward_mean", mode="max")
print(log_dir)

/home/dejang/ray_results/PPOTrainer_2022-06-29_14-54-26/PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39


In [15]:

# wandb.tensorboard.patch(root_logdir=log_dir, save=False, tensorboard_x=True)
# If running in a notebook, finish the wandb run to upload the tensorboard logs to W&B
wandb.finish()


"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/global_step",▂▃▃▄▅▆▆▇█▁
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/agent_timesteps_total",▁▂▃▃▄▅▆▆▇█
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/counters/num_agent_steps_sampled",▁▂▃▃▄▅▆▆▇█
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/counters/num_agent_steps_trained",▁▂▃▃▄▅▆▆▇█
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/counters/num_env_steps_sampled",▁▂▃▃▄▅▆▆▇█
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/counters/num_env_steps_trained",▁▂▃▃▄▅▆▆▇█
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/done",▁▁▁▁▁▁▁▁▁█
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/episode_len_mean",▁▁▁▁▁▁▁▁▁
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/episode_reward_max",▁▁▆▆▇▇███
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/episode_reward_mean",▁▁▅▅▇▇███
"PPOTrainer_compiler_gym_e618f_00001_1_0=40,1=20_2022-06-29_14-54-39/ray/tune/episode_reward_min",▁▁▁▁▁▁▁▁▁


In [17]:
trial.stopping_criterion

{'episodes_total': 5}

In [18]:
trial.metric_analysis

{'episode_reward_max': {'max': -2032655828.5176554,
  'min': -2113860984.255222,
  'avg': nan,
  'last': -2032655828.5176554,
  'last-5-avg': -2037221708.413356,
  'last-10-avg': nan},
 'episode_reward_min': {'max': -2113860984.255222,
  'min': -2113860984.255222,
  'avg': nan,
  'last': -2113860984.255222,
  'last-5-avg': -2113860984.2552218,
  'last-10-avg': nan},
 'episode_reward_mean': {'max': -2058342156.6313508,
  'min': -2113860984.255222,
  'avg': nan,
  'last': -2058342156.6313508,
  'last-5-avg': -2063631441.049586,
  'last-10-avg': nan},
 'episode_len_mean': {'max': 10.0,
  'min': 10.0,
  'avg': nan,
  'last': 10.0,
  'last-5-avg': 10.0,
  'last-10-avg': nan},
 'episodes_this_iter': {'max': 1,
  'min': 0,
  'avg': 0.5,
  'last': 1,
  'last-5-avg': 0.6,
  'last-10-avg': 0.5},
 'num_healthy_workers': {'max': 1,
  'min': 1,
  'avg': 1.0,
  'last': 1,
  'last-5-avg': 1.0,
  'last-10-avg': 1.0},
 'num_agent_steps_sampled': {'max': 50,
  'min': 5,
  'avg': 27.5,
  'last': 50,
  'l

In [19]:
!rllib rollout ~/ray_results/PPOTrainer_2022-06-29_12-56-45/PPOTrainer_compiler_gym_7508e_00000_0_2022-06-29_12-56-45/checkpoint_000010/checkpoint-10 --run PPO --episodes 5

2022-06-29 14:56:06,383	WARNING deprecation.py:46 -- DeprecationWarning: `rllib rollout` has been deprecated. Use `rllib evaluate` instead. This will raise an error in the future!
2022-06-29 14:56:08,423	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8267
2022-06-29 14:56:09,753	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-06-29 14:56:09,754	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-06-29 14:56:09,754	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
Traceback (most recent call last):
  File "/home/dejang/anaconda3/

In [20]:
ray.shutdown()

In [21]:
# agent.restore(checkpoint)

In [22]:
# def run_agent_on_benchmarks(benchmarks):
#     """Run agent on a list of benchmarks and return a list of cumulative rewards."""
#     with make_env() as env:
#         rewards = []
#         for i, benchmark in enumerate(benchmarks, start=1):
#             observation, done = env.reset(benchmark=benchmark), False
#             while not done:
#                 action = agent.compute_single_action(observation)
#                 observation, _, done, _ = env.step(int(action))
#             rewards.append(env.episode_reward)
            
#             print(f"[{i}/{len(benchmarks)}] ")

#     return rewards

In [23]:
# train_rewards = run_agent_on_benchmarks(train_benchmarks)
# test_rewards = run_agent_on_benchmarks(test_benchmarks)


In [24]:
# from matplotlib import pyplot as plt
# import numpy as np

In [25]:
# fig, axs = plt.subplots(1, 2)

# axs[0].title.set_text('Train rewards')
# axs[0].plot(train_rewards, color="red")
# axs[0].plot(np.zeros_like(train_rewards), color="blue")

# axs[1].title.set_text('Test rewards')
# axs[1].plot(test_rewards, color="green")
# axs[1].plot(np.zeros_like(test_rewards), color="blue")

# plt.tight_layout()
# plt.show()
